In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

We will read the input data voice.csv using read_csv and check the top 5 rows from the data

In [ ]:
data = pd.read_csv('../input/voice.csv')
data.head(5)

In [ ]:
data.label.unique

In [ ]:
print(data.info())

We will code "female" as 1 and "male" as 0.

In [ ]:
data.label = [1 if each=="female" else 0 for each in data.label]

In [ ]:
data.label.values

We will have the "label" values into 'y' and drop the "label" column and have 'x_data'

In [ ]:
y = data.label.values
x_data = data.drop(['label'],axis=1)

We will now normalize the data

In [ ]:
np.min(x_data)

In [ ]:
np.max(x_data)

Normalization logic: 
(x- min(x))/(max(x)-min(x))

In [ ]:
x = (x_data - np.min(x_data))/(np.max(x_data)-np.min(x_data)).values

In [ ]:
x.head(5)

In [ ]:
x_data.head(5)

Now, we will split the dataset into train and test

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

#Now lets transpose the above all
x_train = x_train.T
x_test = x_test.T
y_train = y_train.T
y_test = y_test.T

print("x_train shape : ", x_train.shape)
print("x_test shape : ", x_test.shape)
print("y_train shape : ", y_train.shape)
print("y_test shape : ", y_test.shape)

We will now create some basic functions for initializing the bias and weights and also a sigmoid function

In [ ]:
def initialization(dimension):
    w = np.full((dimension,1), 0.01)
    b = 0.0
    return w,b

def sigmoid(z):
    y_head = 1/(1+np.exp(-z))
    return y_head


Now, we will create a function for forward and backward propagation

In [ ]:
def f_b_propagation(w,b,x_train, y_train):
    #forward propagation
    z = np.dot(w.T, x_train) + b
    y_head = sigmoid(z)
    loss = -( y_train * np.log(y_head)- ( 1 - y_train ) * np.log( 1 - y_head))
    cost = (np.sum(loss))/x_train.shape[1]   # x_train.shape[1]is for scaling the cost
    
    #backward propagation
    derivative_weight = (np.dot(x_train, ((y_head - y_train).T)))/x_train.shape[1]
    #x_train.shape[1] is for scaling purpose
    
    derivative_bias = np.sum(y_head - y_train)/ x_train.shape[1]
    #x_train.shape[1] is for scaling purpose
    
    gradients = {"Derivative_Weight": derivative_weight, "Derivative_Bias": derivative_bias}
    
    return cost, gradients

Now, we will write a function to update the parameters


In [ ]:
def update(w, b, x_train, y_train, learning_rate, number_of_iterations):
    cost_list_1 = []
    cost_list_2 = []
    index = []
    
    #Iterate over the updating of parameters
    for i in range(number_of_iterations):
        #Make forward and backward propagation to find the cost and gradient
        cost, gradients = forward_backward_propagation(w, b, x_train, y_train)
        cost_list_1.append(cost)
        
        w = w - learning_rate * gradients["derivative_weight"]
        b = b - learning_rate * gradients["derivative_bias"]
        
        if i % 10 == 0:
            cost_list_2.append(cost)
            index.append(i)
            print("Cost after iteration %i =  %f " %(i, cost))
        